# A descriptive optimization model for epidemic scenarios (SARS-CoV-2) exploiting the SIR compartmental model and Lagrangian Duality

In [27]:
# ## Imports

# Data management and manipulation
import pandas as pd
from urllib.request import urlretrieve
import datetime # dates
from util.SIRModel.reader import download_data, read_SIR

# Model
from util.SIRModel.model import SIRModel, save_parameters

# Plotting
from util.SIRModel.plotter import plotter, plot_loss_history, plot_pars, plot_SIR#, plot_prediction, plot_RK

from tqdm import tqdm # nice progress bars

%matplotlib widget
%load_ext autoreload
%autoreload 2


repository = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/'
url = 'dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv'

pop_aug = 59392408 # Italy population at 1.08.2020
pop_sep = 59376037 # Italy population at 1.09.2020
N = round((pop_aug+pop_sep)/2) # Italy population estimate at 15.08.2020

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Outline <br/><br/>


* Goal
* Modelling the problem
    - Available data
    - SIR epidemiological model
    - Lagrangian Duality for Constrained Machine Learning
    - Implementation
* Data preprocessing
* Model definition
* Training
* Results

## Goal <br/><br/>

- The aim of the present work is to provide a **descriptive model for epidemic scenarios**
- In particular, we want to train an *optimization algorithm* on currently available data so as to
  - **estimate the <b>parameters</b> of the epidemic**
  - **reproduce the <b>evolution</b> of the epidemic**, with focus on the infected individuals.
- We exploit a **hybrid** approach, combining **model-driven** and **data-driven** techniques, as we will see shortly.

## Modelling the problem

### Available data<br/><br/>

First of all, we need to figure out how available data is structured, in order to design our model.

- We rely on the official [COVID-19](https://github.com/pcm-dpc/COVID-19) data repository — provided by the Presidenza del Consiglio dei Ministri (specifically, by the Protezione Civile department)
  - it is organized in different sections according to different aggregation levels: national, regional, provincial.
  - Each section consists of `.csv` files, one for each day, containing tabular data: in particular, we focused on national data, thus inspecting the `dati-andamento-nazionale` section.
  - The most interesting columns of these files are `['totale_positivi', 'dimessi_guariti', 'deceduti']`, since they represent the core information about the pandemic: the number of infected, of healed, and of deceased people.
- As we see in what follows, this data allows us to build a model to describe the trend of the epidemic.

### Inspecting data<br/><br/>

- Let's download raw data from the correspondent [repository](https://github.com/pcm-dpc/COVID-19/tree/master/dati-andamento-nazionale) and import it into a `pandas.DataFrame` object to inspect it

In [28]:
df = pd.read_csv(repository+url)
df.head()

,data,stato,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,...,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,ITA,101,26,127,94,221,0,221,1,...,4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-25T18:00:00,ITA,114,35,150,162,311,90,93,1,...,8623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-26T18:00:00,ITA,128,36,164,221,385,74,78,3,...,9587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-27T18:00:00,ITA,248,56,304,284,588,203,250,45,...,12014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-28T18:00:00,ITA,345,64,409,412,821,233,238,46,...,15695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### The SIR epidemiological model<br/><br/>

The simplest *compartmental model* to describe the spread of infectious diseases is the **SIR model** without vital dynamics — i.e. in which birth and death rates are neglected — which can be expressed through the following set of ordinary differential equations (ODEs):

$$
\begin{cases}
{\displaystyle \frac{dS}{dt} = \frac{-\beta I S}{N}} \\
{\displaystyle \frac{dI}{dt} = \frac{\beta I S}{N} - \gamma I} \\
{\displaystyle \frac{dR}{dt} = \gamma I} \\
\end{cases}
$$

- $N$ is the total population
- $S$ is the *compartment* of **S**usceptible (i.e. healthy), $I$ of **I**nfected (diseased and infectious) and $R$ of **R**ecovered (either healed and immunized or deceased) individuals
  - Note that $S$, $I$ and $R$ are mutually exclusive, so that $N=S+I+R$
- $\beta>0$ is the <b>infection rate</b><sup>1</sup>, and $0<\gamma<1$ is the <b>recovery rate</b><sup>2</sup>.

<br><p class="tiny"><sup>1</sup>Defined as the <em>average number of contacts</em> per person per time, multiplied by the <em>probability of disease transmission</em> in a contact between a susceptible and an infectious subject.<br>
<sup>2</sup>Defined as the inverse of the degency period, i.e. the time that an individual stays in the $I$ compartment (before moving to $R$).</p>

### The SIR epidemiological model<br/><br/>

- (Almost) everything we need is stored in the data:
  - $I$ compartment data corresponds to column `'totale_positivi'`
  - $R$ data can be computed as the sum of the columns `'dimessi_guariti', 'deceduti'`
  - Recalling that $N = S + I + R$, we collect the total population $N$ from [Istat](http://demo.istat.it/bilmens/index.php?anno=2020&lingua=ita) data and compute $S$ as $N - I - R$
- We can now tackle the problem — as mentioned before — using a *hybrid* approach, consisting of:
  * A **model-driven** approach: the set of ODEs above can be numerically solved through iterative methods such as the [Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (in particular, the first-order one, also known as [Euler method](https://en.wikipedia.org/wiki/Euler_method), and the fourth-order one, also referred to as [RK4](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods#Derivation_of_the_Runge%E2%80%93Kutta_fourth-order_method). Indeed, they allow to numerically compute the evolution of the variables $S(t)$, $I(t)$, $R(t)$, given the initial state $S(0)$, $I(0)$, $R(0)$ and the parameters $\beta$, $\gamma$.
  * A **data-driven** approach: conversely — given the evolution of $S(t)$, $I(t)$ and $R(t)$ as both data and predictions — we could train an *optimization algorithm* to estimate the best fitting parameters $\beta$ and $\gamma$.

### The SIR epidemiological model<br/><br/>

- We can now observe that $\gamma$ tends to keep a *roughly constant* value during an epidemic, whereas $\beta$ does not:
  * the <b>spread rate</b> of the disease is mainly influenced by continuously floating variables — such as **containment measures** and ultimately **people behaviour**
  * the <b>degency period</b>, on the other hand, is affected only by clinical and treatment issues, which are much less prone to variations.
- Hence, we can simplify our problem assigning a **constant value to $\gamma$**<sup>3</sup> and performing the optimization **only over $\beta$**.
- Conversely, we express the latter as a time-dependent variable $\beta\left(t\right)$ so as to capture its significant variability
  - Thus, we will more appropriately refer to it as a vector of parameters $\boldsymbol{\beta}=(\beta_{0},...,\beta_{T-1})$ 
  - one for each day $t$ with $\beta_{t}\equiv\beta\left(t\right)$
  - with $T$ the total number of days covered by available data.

<br><p class="tiny"><sup>3</sup>We used the value $\gamma = 1/17$, which corresponds to a degency period of 17 days, obtained by global trends.</p>

### The SIR epidemiological model<br/><br/>

* The characteristic evolution of a SIR model without vital dynamics

<center><div style="background-image:url('assets/SIR_model.png'); width:600px; height:300px; background-position:center; background-repeat: no-repeat;"></div></center>

### Inspecting data<br/><br/>

* Let's take a look to data we are interested in
* We can notice similarities with the SIR model:
  - the `'totale_positivi'` resembles the $I$ compartment
  - the `'dimessi_guariti'` resembles the $R$ compartment

In [48]:
labels = ['totale_positivi', 'dimessi_guariti', 'deceduti']
plotter(df[labels].values.T, labels=labels)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Data preprocessing<br/><br/>

* Data is already pretty clean
  * We need to filter the dataframe to keep only useful information
  * For convenience, we parse dates setting a `pd.DatetimeIndex`

```python
def download_data(url, ...):
    ...
    dataframe = pd.read_csv(url)
    ...
    # Select interesting columns
    dataframe = dataframe.filter(['data', 'totale_positivi', 'dimessi_guariti', 'deceduti'])
    # Parse dates
    dataframe['data'] = pd.to_datetime(dataframe['data'], format='%Y-%m-%d').dt.date
    # Set DatetimeIndex
    dataframe = dataframe.set_index(pd.DatetimeIndex(dataframe['data'])).drop('data', axis=1)
    return dataframe
```

In [49]:
dataframe = download_data(repository+url)

### Retrieve input data<br/><br/>

We now select a time interval and extract the corresponding S,I,R variables

In [50]:
start_date=datetime.date(2020, 8, 15)
end_date=datetime.date(2021, 5, 1)

# Read values
S, I, R = read_SIR(dataframe, start_date, end_date, N)

## Retrieve input data

```python
# Retrieve S,I,R from Dataframe and eventually add padding
def read_SIR(dataframe, start_date, end_date, N, ...)
    # Select dates
    dataframe = dataframe.loc[start_date:end_date]
    # Retrieve S, I, R from DataFrame
    I = np.array(dataframe['totale_positivi'], dtype='float64')
    recovered = np.array(dataframe['dimessi_guariti'], dtype='float64')
    deceased = np.array(dataframe['deceduti'], dtype='float64')
    R = recovered + deceased
    S = N - I - R
    ...
    return S, I, R
```

## Baseline

### Baseline definition <br/><br/>

* First of all, we define a simple **baseline model** to approach the problem
* We start by defining a **loss function**:
$$
f_{\boldsymbol{\beta}}\left(I,R\right) = \frac{1}{T}\sum_{t=0}^{T-1}{\left(\hat{I}_{t}-I_{t}\left(\boldsymbol{\beta}\right)\right)^{2} + \epsilon_{R}\left(\hat{R}_{t}-R_{t}\left(\boldsymbol{\beta}\right)\right)^{2}}
$$
- Basically, the sum of the **Mean Squared Errors** on both the $I$ and the $R$ compartments, this latter down-weighted by a constant $\epsilon_{R} < 1$
  * both terms depend only on $\boldsymbol{\beta}$, which indeed is our only trainable parameter.
- Our goal is obtaining the **parameters $\boldsymbol{\beta}$** that **best fit the available data**

### Baseline definition<br/><br/>

We come now to the actual model definition, wrapped inside the `SIRModel` class:

```python
class SIRModel():
    # Model creation
    def __init__(self, ...):
        ...
    # Approximation
    def run_simulation(self):
        ...
    # Train: minimize loss
    def train(self, ...):
        ...
```

### Baseline definition
```python
def __init__(self, S, I, R, N, gamma=1./17, eps_R=1e-3):
    ...
    # Parameters
    self.initializer_beta = tf.keras.initializers.RandomUniform(minval=0, seed=42)
    self.betas = tf.Variable(self.initializer_beta(shape=(self.T,), dtype='float64'), trainable=True)
    # Weight
    self.eps_R = tf.constant(eps_R, shape=(), dtype='float64')
    # Optimizer
    self.optimizer = tf.keras.optimizers.Adam()
    # Loss history
    self.loss_history = tf.Variable([], dtype='float64')
```

### Baseline definition
```python
def __step(self, t):
    S_to_I = (self.betas[t] * self.I_[t] * self.S_[t]) / self.N
    I_to_R = self.gammas[t] * self.I_[t]
    self.S_ = tf.concat([self.S_, [self.S_[t] - S_to_I]], 0)
    self.I_ = tf.concat([self.I_, [self.I_[t] + S_to_I - I_to_R]], 0)
    self.R_ = tf.concat([self.R_, [self.R_[t] + I_to_R]], 0)
def run_simulation(self):
    self.S_ = tf.Variable([self.S[0]], dtype='float64')
    self.I_ = tf.Variable([self.I[0]], dtype='float64')
    self.R_ = tf.Variable([self.R[0]], dtype='float64')
    for t in range(self.T-1):
        self.__step(t)
```
* The *Euler's method* is used to compute the evolution of $S(t)$, $I(t)$ and $R(t)$, given the initial values $S(0)$, $I(0)$ and $R(0)$ along with the parameters $\boldsymbol{\beta}$

### Baseline definition
```python
def loss_fn(self):
    self.run_simulation()
    # Error on I
    err_I = K.mean(K.square(self.I_ - self.I))
    # Error on R
    err_R = self.eps_R * K.mean(K.square(self.R_ - self.R))
    loss = err_I + err_R
    return loss
def train(self, epochs, ...):
    for _ in tqdm(range(epochs)):
        self.optimizer.minimize(self.loss_fn, var_list=[self.betas])
```
* Training loop: at each step, this method simply performs a gradient update towards the minimization of the value returned by `loss_fn()` through `optimizer.minimize`

### Training the baseline

In [32]:
epochs = 300
eps_R=1e-3
gamma = 1./17

baseline =  SIRModel(S, I, R, N, gamma=gamma, eps_R=eps_R)

In [33]:
baseline.train(epochs)

100%|██████████| 300/300 [05:57<00:00,  1.19s/it]


### Inspecting results<br/><br/>

* Let's firstly inspect the loss variation
* The loss decreases quite regularly
  * Apart from a small bump

In [51]:
plot_loss_history(baseline)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Inspecting results<br/><br/>

* Let's see the predictions for $\beta$
* The prediction is **very noisy**
* However, a trend is clearly visible

In [52]:
plot_pars(baseline, start_date)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Inspecting results<br/><br/>

* Let's run the simulation to obtain SIR data from fitted parameters
* Actually, we are interested only in the $I$ compartment, since it raises the major issues
* Running the SIR simulation actually produces decent results, not far from real data
  * Employing both Euler's method (better) and RK4
* Still, we have a jagged behaviour on $I$
* Besides, we have no trustworthy estimation of $\beta$
  * We would like to have a clearer, *smoother* behaviour

In [53]:
plot_SIR(baseline, I, start_date)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

R2 scores: ['0.99 (E)', '0.98 (RK)']
mse:['5.43e-04 (E)', '1.47e-03 (RK)']


## An improvement: Semantic Based Regularization

### Smoothness through regularization<br/><br/>

- To achieve meaningful and realistic results — and also to reasonably predict the evolution of the epidemic for short periods of time — we would like to enforce a smooth behaviour on $\beta\left(t\right)$, rather than an irregular one which presents sudden changes.
- Provided that we choose a tolerance threshold $\Delta\beta$, we can impose a constraint on the problem of the form

$$
\mid\beta_{t}-\beta_{t-1}\mid\ \leq\ \Delta\beta \ \ \ \forall t \in \{1,...,T-1\},
$$

- we can use the constraint to derive a **Semantic Based Regularizer** (SBR)
  - even though it requires us to *tune the weight hyperparameter*

### SBR model definition <br/><br/>

Accordingly, we update the **loss function** for our model:
$$
f_{\boldsymbol{\beta}}\left(I,R\right) = \frac{1}{T}\sum_{t=0}^{T-1}{\left(\hat{I}_{t}-I_{t}\left(\boldsymbol{\beta}\right)\right)^{2} + \epsilon_{R}\left(\hat{R}_{t}-R_{t}\left(\boldsymbol{\beta}\right)\right)^{2}} + \lambda\frac{1}{T-1}\sum_{t=1}^{T-1}{\left(\beta_{t+1}-\beta_{t}\right)^{2}}
$$
where
- the first term is the usual sum of the **Mean Squared Errors** on both the $I$ and the $R$ compartments
- the second term is the **regularization** over $\beta\left(t\right)$:
  * the sum term represents the violation of the smoothness constraint
  * the coefficient $\lambda$ *weights* the violation of the smoothness constraint

This loss function reflects our goals:
* Obtain the **parameters $\boldsymbol{\beta}$** that **best fit the available data** ...
* ... enforcing a trend for $\beta\left(t\right)$ **as smooth as possible**.

### SBR model definition<br/><br/>

* Let's implement the changes into the model

```python
def __init__(self, S, I, R, N, gamma=1./17, eps_R=1e-3, lambda_beta=0):
    ...
    # Parameters
    self.initializer_beta = tf.keras.initializers.RandomUniform(minval=0, seed=42)
    self.betas = tf.Variable(self.initializer_beta(shape=(self.T,), dtype='float64'), trainable=True)
    # Weights
    self.eps_R = tf.constant(eps_R, shape=(), dtype='float64')
    self.lambda_beta = tf.Variable(lambda_beta, dtype='float64')
    # Optimizer
    self.optimizer = tf.keras.optimizers.Adam()
    # Loss history
    self.loss_history = tf.Variable([], dtype='float64')
    self.mse_history = tf.Variable([], dtype='float64')
    self.cst_history = tf.Variable([], dtype='float64')
```

### SBR model definition
```python
def loss_fn(self):
    self.run_simulation()
    # Error on I
    err_I = K.mean(K.square(self.I_ - self.I))
    # Error on R
    err_R = self.eps_R * K.mean(K.square(self.R_ - self.R))
    mse = err_I + err_R
    # Regularization on beta
    self.beta_cst = K.mean(K.square(self.betas[:-1] - self.betas[1:]))
    cst = self.lambda_beta * self.beta_cst
    loss = mse + cst
    # Update loss history
    self.loss_history = tf.concat([self.loss_history, [loss]], 0)
    self.mse_history = tf.concat([self.mse_history, [mse]], 0)
    self.cst_history = tf.concat([self.cst_history, [cst]], 0)
    return loss
```

### Training the SBR model

In [37]:
epochs = 300
eps_R=1e-3
gamma = 1./17

sbr =  SIRModel(S, I, R, N, gamma=gamma, eps_R=eps_R, lambda_beta=1e13)

In [38]:
sbr.train(epochs)

100%|██████████| 300/300 [05:57<00:00,  1.19s/it]


### Inspecting results<br/><br/>

* The loss function decreases, as before
  * However, we reach a greater absolute value with respect to the baseline
  * Due to the *conflicting goals* of the two terms
* We can notice the relevance of the constraint on the total loss
  * Seems quite negligible, at least by inspecting the plot

In [54]:
plot_loss_history(sbr, reg=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Inspecting results<br/><br/>

* **Much of the noise** is gone, although some "ups and downs" still persist
* This is the clearest evidence that the regularization is working

In [55]:
plot_pars(sbr, start_date)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Inspecting results<br/><br/>

* The SIR simulation produces better results, in particular we obtain a more regular trend
  * Once again, employing both Euler's method (still better) and RK4
* However, we needed to tune another hyperparameter
  * To find the right trade-off between MSE and regularization
* Besides, the resulting prediction still presents irregularities
  * It tends to stay close to true data, at the expense of smoothness

In [56]:
plot_SIR(sbr, I, start_date)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

R2 scores: ['0.99 (E)', '0.98 (RK)']
mse:['1.19e-03 (E)', '1.69e-03 (RK)']


## A recent approach: Lagrangian Duality for Constrained Learning

### Smoothness through regularization... again!<br/><br/>
<blockquote>
<h3> Smoothness through regularization<br/><br/></h3>
    
- To achieve meaningful and realistic results — and also to reasonably predict the evolution of the epidemic for short periods of time — we would like to enforce a smooth behaviour on $\beta\left(t\right)$, rather than an irregular one which presents sudden changes.
- Provided that we choose a tolerance threshold $\Delta\beta$, we can impose a constraint on the problem of the form
    
$$
\mid\beta_{t}-\beta_{t-1}\mid\ \leq\ \Delta\beta \ \ \ \forall t \in \{1,...,T-1\},
$$
    
    
- we could use the constraint to derive a **Semantic Based Regularizer** (SBR)
  - even though it would require us to *tune the weight hyperparameter*

</blockquote>

- or we could exploit a [recent approach](https://arxiv.org/pdf/2001.09394.pdf) to address this problem in a more robust and simplified fashion, as discussed in what follows.

### Lagrangian Duality for Constrained Learning <br/><br/>

As recalled in the paper, when dealing with an optimization problem of the form
$$
\cal O = \underset{y}{\operatorname{argmin}} f\left(y\right) \ \mathrm{\text{ subject to }}\ g_{i}\left(y\right)\leq0 \ \left(\forall i \in \left[n\right]\right),
$$
the use of (non-negative) *Lagrangian multipliers* allows to rewrite the objective function in its *Lagrangian relaxed form*, which ultimately **translates the constraints into loss penalties** for violating them.
As the paper points out, the original problem then becomes
$$
f_{\boldsymbol{\lambda}}\left(y\right) = f\left(y\right) + \sum_{i=1}^{n}{\lambda_{i}g_{i}\left(y\right)} \\
$$
where $\boldsymbol{\lambda} = \left(\lambda_{1}, ..., \lambda_{n}\right)$ is the vector of the multipliers, with $\lambda_{i}\geq0$, while $g_{i}\left(y\right)$ are the penalty constraints (also expressed as $\max\left(0,g_{i}\left(y\right)\right)$ in some formulations, to express the *non-negative* quantification of the violation).
The problem amounts now to the optimization of the Lagrangian relaxation:
$$
LR_{\boldsymbol{\lambda}} = \underset{y}{\operatorname{argmin}} f_{\boldsymbol{\lambda}}\left(y\right)
$$

### Lagrangian Duality for Constrained Learning <br/><br/>

Keeeping in mind that the relaxation is by construction a lower bound for the original problem:
$$
f\left(LR_{\boldsymbol{\lambda}}\right) \leq f\left(\cal O\right),
$$
we can consider an upper bound for it to get the *strongest relaxation* of the problem:

$$
LD = \underset{\lambda_{i} \geq 0}{\operatorname{argmax}} f\left(LR_{\boldsymbol{\lambda}}\right).
$$

- This latter problem is known as **Lagrangian dual** and it can be used to find the best *Lagrangian multipliers*.
- The power of this method relies in turning the original problem into an *easier one*, at the same time enforcing a *strong approximation* of it.
- Besides, it provides a further benefit with respect to the SBR approach:
  - Within an SBR approach, the weight of the regularization is a *hyperparameter* to tune
  - Within a Lagrangian dual approach, the weight (multiplier) is a *trainable variable*!

### Lagrangian Duality: implementation <br/><br/>

- Intuitively, we need a loop that iteratively seeks to first **minimize** the Lagrangian obtaining a class of *candidate lower bounds* for it, and then looks for the *greatest* candidate as the **tightest lower bound**.
- The formal definition of the problem can be thus approximately translated into a loop which, for each optimization step $k$:
  - Firstly performs a gradient step over $y$ so as to *minimize* the relaxed Lagrangian $f_{\boldsymbol{\lambda}}\left(y\right)$, obtaining the value $y_{\boldsymbol{\lambda}, k}$;
  - Then performs another gradient step over each $\lambda_{i}$ so as to *maximize* $f\left(y_{\boldsymbol{\lambda}, k}\right)$.

### LD model definition <br/><br/>

Recalling the **loss function** for our model:
$$
f_{\boldsymbol{\beta}}\left(I,R\right) = \frac{1}{T} \left(\sum_{t=0}^{T-1}{\left(\hat{I}_{t}-I_{t}\left(\boldsymbol{\beta}\right)\right)^{2} + \epsilon_{R}\left(\hat{R}_{t}-R_{t}\left(\boldsymbol{\beta}\right)\right)^{2}}\right) + \lambda_{\boldsymbol{\beta}}\frac{1}{T-1}\left(\sum_{t=1}^{T-1}{\left(\beta_{t+1}-\beta_{t}\right)^{2}}\right),
$$

Recalling the discussion about *Lagrangian Duality:*
- $\boldsymbol{\beta}$ is our $y$, while our (only) <b>Lagrangian multiplier</b> is $\lambda_{\boldsymbol{\beta}}$ (now a *trainable* parameter!)
- we would like to firstly **_minimize the loss function_** over $\boldsymbol{\beta}$
    - to do this, we take both MSE and regularization into account
- then, exploiting Lagrangian Duality, we would like to **_maximize the loss function_** over the Lagrangian multiplier $\lambda_{\boldsymbol{\beta}}$
    - to do this, we update the Lagrangian multiplier at each step according to a Lagrangian step size $\mu_{\beta}$

### LD model definition<br/><br/>

* Let's apply the changes to our model accordingly:

```python
def __init__(self, S, I, R, N, gamma=1./17, eps_R=1e-3, lambda_beta=0, mu_beta=0):
    ...
    # Parameters
    self.initializer_beta = tf.keras.initializers.RandomUniform(minval=0, seed=42)
    self.betas = tf.Variable(self.initializer_beta(shape=(self.T,), dtype='float64'), trainable=True)
    ...
    # Lagrangian dual parameters
    self.mu_beta = mu_beta
    self.lambda_beta = tf.Variable(lambda_beta, dtype='float64')
    self.beta_cst = tf.Variable(0, dtype='float64')
    # Weight
    self.eps_R = tf.constant(eps_R, shape=(), dtype='float64')
    # Optimizer
    self.optimizer = tf.keras.optimizers.Adam()
    ...
```

### LD model definition
```python
def optimize_lagrangian_parameters(self):
    self.lambda_beta = self.lambda_beta + self.mu_beta * self.beta_cst
def train(self, epochs, ldf=True):
    for _ in tqdm(range(epochs)):
        self.optimizer.minimize(self.loss_fn, var_list=[self.betas])
        if ldf:
            self.optimize_lagrangian_parameters()
```
* Training loop: at each step, this method firstly performs a gradient update towards the minimization of the value returned by `loss_fn()` through `optimizer.minimize`, then another (custom) one towards the maximization of $\lambda_{\beta}$, performed as follows:
$$
\lambda_{\beta_{k+1}} = \lambda_{\beta_{k}} + \mu_{\beta_{k}}\frac{1}{N-1}\sum_{t=1}^{N-1}{\left(\beta_{k_{t+1}}-\beta_{k_{t}}\right)^{2}}
$$
where the sum quantifies the violation of the **smoothness constraint**, while the Lagrangian step size $\mu_{\beta}$ acts as a *learning rate coefficient* for the sum term.

```python
def predict(self, days):
    self.run_simulation()
    S = [self.S_[-1].numpy()]
    I = [self.I_[-1].numpy()]
    R = [self.R_[-1].numpy()]
    N = self.N.numpy()
    
    # Compute beta: exponentially smoothed average of last 5 days
    beta = self.betas[-5:]
    beta_mean = tf.reduce_mean(beta)
    soft = tf.nn.softmax(tf.linspace(0., len(beta)-1, len(beta))).numpy()
    beta_avg = tf.reduce_mean((beta-beta_mean)*soft +beta_mean)
    
    gamma = self.gammas[0].numpy()
    for day in range(days):
        S_to_I = (beta_avg * I[-1] * S[-1]) / N
        I_to_R = I[-1] * gamma
        S.append(S[-1] - S_to_I)
        I.append(I[-1] + S_to_I - I_to_R)
        R.append(R[-1] + I_to_R)

        return S, I, R, beta_avg.numpy()
```

* Employ the last values obtained by the simulation ($S_{T-1}$, $I_{T-1}$, $R_{T-1}$<sup>4</sup>) as initial point to run a simulation of `days` steps; the same parameter $\beta_{p}$ was used for every step, obtained as  as follows:
$$
\beta_{p} = \frac{1}{n}\sum_{t=0}^{n-1}{\left(\beta_{T-t-1}-\bar{\beta}\right)\sigma\left(t\right)+\bar{\beta}}\ \ \ \ \ \ \mathrm{\text with}\ \ \sigma\left(t\right) = \frac{e^{t}}{\sum_{t=0}^{n-1}{e^{t}}}
$$
where we used the *softmax function* $\sigma\left(t\right)$ and assigned $n=5$. This is a very simple method to obtain a rough approximation for $\beta_{p}$: it basically computes an exponentially-smoothed weighted average of the last $n$ known values of $\beta_{t}$, assigning a gradually *lower relevance to older values*.

### Training the LD model

In [42]:
epochs = 300
eps_R=1e-3
mu_beta=1e16
gamma = 1./17

lgd = SIRModel(S, I, R, N, gamma=gamma, eps_R=eps_R, lambda_beta=0, mu_beta=mu_beta)

### Training the LD model

In [43]:
lgd.train(epochs, ldf=True)

100%|██████████| 300/300 [06:04<00:00,  1.21s/it]


### Inspecting results<br/><br/>

* We sacrificed another (small) piece of loss in favour of the regularization
* Again, the regularization loss accounts for an (apparently) negligible part of the main loss

In [57]:
plot_loss_history(lgd, reg=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fitted parameters<br/><br/>

* Take a look at the fitted parameters to evaluate the regularization
* We can notice a **very smooth** behaviour

In [58]:
plot_pars(lgd, start_date)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fitted SIR<br/><br/>

* As a result, we can enjoy a good fit
    * Both accurate and smooth
    * Euler's method still tends to perform better than RK4
* We can notice a (slight) increase in the $MSE$ with respect to the SBR model
  * The LD model ended up *sacrificing* another piece of prediction quality for the sake of smoother parameters

In [59]:
plot_SIR(lgd, I, start_date)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

R2 scores: ['0.99 (E)', '0.98 (RK)']
mse:['1.31e-03 (E)', '1.73e-03 (RK)']


In [47]:
#save_parameters(model)

## Conclusions

### Conclusions<br/><br/>

* We combined a model-driven approach based on ODEs and a data-driven approach based on Machine Learning
* We employed Lagrangian Duality to **inject constraints** into the model
* We **fitted** COVID-19 data to obtain the **parameters** of the epidemic
* We ended up with a **descriptive model** for estimating the parameters of the epidemic

### Future work<br/><br/>

* Now, if we have NPI (Non Pharmaceutical Interventions) data, it is possible to exploit our results to build a **predictive model**:
  - Whose _input features_ are the NPIs
  - Whose _target_ is fitted data ($\beta$ parameters)
* If we can manage to model the relationship between NPIs and $\beta$, we could **predict** the evolution of the epidemic (in particular, of the infected)
* It would then be possible to build a **prescriptive model**, namely a model to establish the best NPI to take in order to obtain a certain evolution of the epidemic.
  - In particular, this could be achieved exploiting the [**Empirical Model Learning**](https://emlopt.github.io/) approach
  - Namely, injecting the ML model into a *Combinatorial optimization model*...
* ... but that is matter for another work!